In [1]:
import processing
import ds
import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearRegression from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVR from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.18 when using versi

In [2]:
filename_train = '/home/octo/Dropbox'+ '/SPY5Dec.csv'

In [3]:
# loading csv file
def get_csv_pd(path):
    #spy_pd=pd.read_csv('C:\\Users\Michal\Dropbox\IB_data\SPY.csv',sep=' ',names=['askPrice','askSize','bidPrice','bidSize'],index_col=0,parse_dates=True)
    #spy_pd=pd.read_csv(path+'\SPY.csv',sep=',',names=['askPrice','askSize','bidPrice','bidSize'],index_col=0,parse_dates=True)
    spy_pd=pd.read_csv(path,sep=',',dtype={'askPrice':np.float32,'askSize':np.float32,
                                           'bidPrice':np.float32,'bidSize':np.float32},index_col=0,parse_dates=True)
    #spy_pd = pd.read_csv(path, usecols=['askPrice','askSize','bidPrice','bidSize'], engine='python', skipfooter=3)
    return spy_pd
def BA(df):
    df.bidPrice=df.loc[:,'bidPrice'].replace(to_replace=0, method='ffill')
    df.bidSize=df.loc[:,'bidSize'].replace(to_replace=0, method='ffill')
    df.askPrice=df.loc[:,'askPrice'].replace(to_replace=0, method='ffill')
    df.askSize=df.loc[:,'askSize'].replace(to_replace=0, method='ffill')
    df=df.dropna()
    return df
def preprocessing_mar(df):
    df=df.dropna()
    # to exclude 0
    #data=data[data['bidPrice']>240]
    #data=data[data['askPrice']>240]
    df=df[df['bidPrice']>df.bidPrice.mean()-df.bidPrice.std()]
    df=df[df['askPrice']>df.askPrice.mean()-df.askPrice.std()]
    df['Open']=(df.askPrice+df.bidPrice)/2
    df['Close']=df.Open.shift(1)
    df['High']=df.askPrice.rolling(10).max()
    df['Low']=df.bidPrice.rolling(10).min()
    df['Volume']=df.askSize+df.bidSize
    vwap=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df['spread']=(df.Close-vwap)
    df['Up']=np.where(np.logical_and(df.Close.diff(12)>0.01,df.Close.diff(1)>0.0),1,0)
    df['Dn']=np.where(np.logical_and(df.Close.diff(12)<-0.01,df.Close.diff(1)<0.0),-1,0)
    df['UD']=np.where(np.logical_and(df.Close.diff(12)>0.01,df.Close.diff(1)>0.0),1,
                     np.where(np.logical_and(df.Close.diff(12)<-0.01,df.Close.diff(1)<0.0),-1,0))
    df=df.dropna()
    return df

In [4]:
#data=processing.get_csv_pd(filename_train)
#data=processing.BA(data)
#data=processing.preprocessing_mar(data)
data=get_csv_pd(filename_train)
data=BA(data)
data=preprocessing_mar(data)
data_km=ml.kalman_ma(data)
data['ckm']=data.Close-data_km
data['rsi']=talib.RSI(np.array(data.Close.astype('float64')))
data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype('float64')),timeperiod=20)
data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=5)
data=data.dropna()
data['ret']=np.log(data.Close/data.Close.shift(12))
data['sc']=np.where(np.logical_and(data.Close/data.Close.shift(12)>1,data.Close.diff(1)>0.0),1,
                     np.where(np.logical_and(data.Close/data.Close.shift(12)<1,data.Close.diff(1)<0.0),-1,0))# above-below. we need to predict this
data=data[38000:45300].dropna()

In [5]:
filename_outlr = 'outlr.sav'
outlr_loaded = pickle.load(open(filename_outlr, 'rb'))
predictions_outlr = outlr_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','High','Low','sc','ckm', 'rsi', 'atr','mom']])
data['outlr']=predictions_outlr

In [6]:
print(data.groupby('outlr').size())

outlr
-1    4381
 1    2919
dtype: int64


In [34]:
hmm_loaded = joblib.load("markov_hmm.pkl")  
predictions_hmm = hmm_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','atr','spread','ckm', 'rsi','mom']])
data['h_state']=predictions_hmm

In [35]:
np.argmin(np.diag(hmm_loaded.transmat_))

1

In [36]:
pd.crosstab(index=data.h_state,columns=data.outlr) 

outlr,-1,1
h_state,,
0,2935,1672
1,105,399
2,1341,848


In [37]:
#data['NAP']=np.where(np.logical_and(data.outlr==-1,np.argmin(np.diag(hmm_loaded.transmat_))==0),-1,1)
data['NAP']=np.where(np.logical_and(data.outlr==-1,data.h_state==1),-1,1)

In [38]:
print(data.groupby('NAP').size())

NAP
-1     105
 1    7195
dtype: int64


In [39]:
print(data.groupby('h_state').size())

h_state
0    4607
1     504
2    2189
dtype: int64


In [40]:
min(data.groupby('h_state').size())

504

## Buy Sell Short points

In [18]:
filename_rf = 'rf.sav'
rf_loaded = pickle.load(open(filename_rf, 'rb'))
predictions_rf = rf_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','Close','spread','ckm', 'rsi', 'atr', 'mom']])

In [23]:
rf_loaded.score(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','Close','spread','ckm', 'rsi', 'atr', 'mom']],data.UD)

0.930958904109589

In [24]:
data['prUD']=predictions_rf

In [25]:
data.groupby('prUD').size()

prUD
-1      13
 0    7287
dtype: int64

In [41]:
pd.crosstab(index=data.NAP,columns=data.prUD) 

prUD,-1,0
NAP,,
-1,8,97
1,5,7190


## Sell and cover points

In [26]:
filename_rf = 'rf-sc.sav'
rf_loaded = pickle.load(open(filename_rf, 'rb'))
predictions_rf = rf_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','ckm','High','Low','rsi', 'atr', 'mom']])

In [28]:
rf_loaded.score(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','ckm','High','Low','rsi', 'atr', 'mom']],data.sc)

0.8913698630136986

In [42]:
pd.crosstab(index=data.NAP,columns=data.sc) 

sc,-1,0,1
NAP,,,
-1,18,85,2
1,381,6447,367


In [30]:
data['prSC']=predictions_rf

In [31]:
pd.crosstab(index=data.NAP,columns=data.prSC) 

prSC,-1,0,1
NAP,,,
-1,251,1171,9
1,278,5130,461
